Temperature scaling for DNABERT model

looks for optimal temperature based on logits and true sequence

In [1]:
import numpy as np
import pandas as pd

from glob import glob

import torch
from torch import nn, optim
from torch.nn import functional as F

In [2]:
data_dir = '/s/project/mll/sergey/effect_prediction/MLM/'

In [6]:
res_path = data_dir + 'clinvar/dnabert/default/preds/'

In [9]:
mapping = {'A':0,'C':1,'G':2,'T':3}

In [10]:
def get_dnabert_logits(res_path):
    '''
    collects logits and sequences
    returns one long sequence and predictions for it
    '''
    logits = []
    targets = []
    for logits_file in glob(res_path + 'logits*.npy'):
        seq_matching = pd.read_csv(logits_file.replace('/logits_','/seq_').replace('.npy','.csv')).set_index('seq_name').squeeze()
        logits.extend(np.load(logits_file))
        targets.extend(seq_matching.values.tolist())
    targets = ''.join(targets)
    targets = list(map(lambda x: mapping[x], targets))
    logits = np.vstack(logits)
    return logits, targets

In [11]:
logits, targets = get_dnabert_logits(res_path)

logits = torch.tensor(logits)
targets = torch.tensor(targets)

In [12]:
temperature = nn.Parameter(torch.ones(1) * 1.5)

In [13]:
optimizer = optim.LBFGS([temperature], lr=0.1, max_iter=500)
nll_criterion = nn.CrossEntropyLoss()

In [14]:
def eval():
    optimizer.zero_grad()
    loss = nll_criterion(logits/temperature, targets)
    loss.backward()
    return loss

optimizer.step(eval)

tensor(593.2281, grad_fn=<NllLossBackward0>)

In [15]:
nll_criterion(logits, targets)

tensor(889.8412)

In [16]:
nll_criterion(logits/temperature, targets)

tensor(1.6082, grad_fn=<NllLossBackward0>)

In [17]:
temperature

Parameter containing:
tensor([9486.8828], requires_grad=True)